In [29]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www.apache.org/dyn/closer.lua/spark/spark-3.5.3/spark-3.5.3-bin-hadoop3.tgz
!tar xf spark-3.5.3-bin-hadoop3.tgz
!pip install -q findspark

zsh:1: command not found: apt-get
tar: Error opening archive: Unrecognized archive format


In [55]:
import os
os.environ["JAVA_HOME"] = "/Library/Java/JavaVirtualMachines/temurin-23.jdk/Contents/Home"
os.environ["SPARK_HOME"] = "'/Users/sudiptabanerjee/Documents/Big Data Project/Chicago-crime-analysis/spark-3.5.3-bin-hadoop3'"


In [67]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

/Users/sudiptabanerjee/Documents/Big Data Project/Chicago-crime-analysis/spark-3.5.3-bin-hadoop3/bin/spark-class: line 71: /Library/Java/JavaVirtualMachines/temurin-8.jdk/Contents/Home/bin/java: No such file or directory
/Users/sudiptabanerjee/Documents/Big Data Project/Chicago-crime-analysis/spark-3.5.3-bin-hadoop3/bin/spark-class: line 97: CMD: bad array subscript
head: illegal line count -- -1


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [57]:
import os
import sys
import platform

# Print system information
print(f"Python Version: {sys.version}")
print(f"Platform: {platform.platform()}")
print(f"Architecture: {platform.machine()}")

# Recommended setup for macOS (Apple Silicon)
os.environ["JAVA_HOME"] = "/Library/Java/JavaVirtualMachines/temurin-8.jdk/Contents/Home"
os.environ["SPARK_HOME"] = "/Users/sudiptabanerjee/Documents/Big Data Project/Chicago-crime-analysis/spark-3.5.3-bin-hadoop3"

# Adjust Python path
sys.path.append(os.path.join(os.environ["SPARK_HOME"], "python"))
sys.path.append(os.path.join(os.environ["SPARK_HOME"], "python/lib/py4j-0.10.9.5-src.zip"))

import findspark
findspark.init()

from pyspark.sql import SparkSession

# Comprehensive Spark Session configuration
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("MyApp") \
    .config("spark.driver.host", "localhost") \
    .config("spark.driver.bindAddress", "127.0.0.1") \
    .config("spark.python.port", "0") \
    .config("spark.driver.extraJavaOptions", "-Dhdp.version=3.0") \
    .config("spark.sql.catalogImplementation", "hive") \
    .getOrCreate()

# Set log level to reduce warnings
spark.sparkContext.setLogLevel("ERROR")

print("Spark Session created successfully!")

Python Version: 3.10.11 (v3.10.11:7d4cc5aa85, Apr  4 2023, 19:05:19) [Clang 13.0.0 (clang-1300.0.29.30)]
Platform: macOS-15.0.1-arm64-arm-64bit
Architecture: arm64


/Users/sudiptabanerjee/Documents/Big Data Project/Chicago-crime-analysis/spark-3.5.3-bin-hadoop3/bin/spark-class: line 71: /Library/Java/JavaVirtualMachines/temurin-8.jdk/Contents/Home/bin/java: No such file or directory
/Users/sudiptabanerjee/Documents/Big Data Project/Chicago-crime-analysis/spark-3.5.3-bin-hadoop3/bin/spark-class: line 97: CMD: bad array subscript
head: illegal line count -- -1


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [58]:
#spark imports
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
import csv
from pyspark.sql.types import *
from pyspark.sql.functions import format_number, when
import pyspark.sql.functions as F

In [66]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

ModuleNotFoundError: No module named 'pandas'

In [ ]:
#test CPU type
import tensorflow as tf
tf.test.gpu_device_name()

In [ ]:
#change RAM allocation, current = 20GB
#change Core allocation, current = 3 Cores

spark = SparkSession.builder.appName("Project - Chicago crime")\
.config("spark.some.config.option", "some-value")\
.config("spark.driver.memory", "20g")\
.config("spark.driver.cores", '3')\
.getOrCreate()

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

In [37]:
crimes_schema = StructType([StructField("ID", IntegerType(), True),
                            StructField("Case Number", StringType(), True),
                            StructField("Date", StringType(), True ),
                            StructField("Block", StringType(), True),
                            StructField("IUCR", StringType(), True),
                            StructField("Primary Type", StringType(), True  ),
                            StructField("Description", StringType(), True ),
                            StructField("Location Description", StringType(), True ),
                            StructField("Arrest", BooleanType(), True),
                            StructField("Domestic", BooleanType(), True),
                            StructField("Beat", IntegerType(), True),
                            StructField("District", IntegerType(), True),
                            StructField("Ward", IntegerType(), True),
                            StructField("Community Area", IntegerType(), True),
                            StructField("FBI Code", StringType(), True ),
                            StructField("X Coordinate", IntegerType(), True),
                            StructField("Y Coordinate", IntegerType(), True ),
                            StructField("Year", IntegerType(), True),
                            StructField("Updated On", StringType(), True ),
                            StructField("Latitude", DoubleType(), True),
                            StructField("Longitude", DoubleType(), True),
                            StructField("Location", StringType(), True )
                            ])

In [ ]:
data = spark.read.csv('/Users/sudiptabanerjee/Documents/Big Data Project/Chicago-crime-analysis/Data-Preprocessing/Crime-1.csv',header = True,schema = crimes_schema)

In [ ]:
data.head(5)

# Data preprocessing - Spark

**Working on data starting 2010.**

In [0]:
dataset = data.filter((data['Year'] >2009) & (data['Year'] !=2020))

In [0]:
#dropping columns which non-relative 

dataset = dataset.drop('ID')
dataset = dataset.drop('Case Number')
dataset = dataset.drop('FBI Code')
dataset = dataset.drop('Updated On')
dataset = dataset.drop('IUCR')# Could be useful - four digit codes that law enforcement agencies use to classify criminal incidents when taking individual reports.
dataset = dataset.drop('X Coordinate')
dataset = dataset.drop('Y Coordinate')
dataset = dataset.drop('Location')
dataset = dataset.drop('Description')
dataset = dataset.drop('Domestic')

**Preprocessing Primary types**

In [0]:
primaryCount = dataset.groupby('Primary Type').count()
primaryCount.orderBy('count', ascending=False).show()

In [0]:
print('Distinct primary types '+ str(primaryCount.distinct().count()))

**Drop crimes which are not so significant**

In [0]:
dataset = dataset.filter((dataset['Primary Type']!='STALKING')&
                             (dataset['Primary Type']!='OBSCENITY')&
                             (dataset['Primary Type']!='NON-CRIMINAL (SUBJECT SPECIFIED)')&
                             (dataset['Primary Type']!='GAMBLING')&
                             (dataset['Primary Type']!='NON - CRIMINAL')&
                             (dataset['Primary Type']!='LIQUOR LAW VIOLATION')&
                             (dataset['Primary Type']!='PUBLIC INDECENCY')&
                             (dataset['Primary Type']!='HUMAN TRAFFICKING')&
                             (dataset['Primary Type']!='INTIMIDATION')&
                             (dataset['Primary Type']!='CONCEALED CARRY LICENSE VIOLATION')&
                             (dataset['Primary Type']!='NON-CRIMINAL')&
                             (dataset['Primary Type']!='OTHER NARCOTIC VIOLATION'))

**Merging similar crime types**

In [0]:
dataset = dataset.withColumn("Primary Type", \
                               when((dataset["Primary Type"] == 'CRIM SEXUAL ASSAULT') | \
                                    (dataset["Primary Type"] == 'PROSTITUTION') ,'SEX OFFENSE').otherwise(dataset['Primary Type']))
dataset=dataset.withColumn("Primary Type", \
                               when((dataset["Primary Type"] == 'WEAPONS VIOLATION') | \
                                    (dataset["Primary Type"] == 'INTERFERENCE WITH PUBLIC OFFICER') ,'PUBLIC PEACE VIOLATION').otherwise(dataset['Primary Type']))
dataset=dataset.withColumn("Primary Type", \
                               when((dataset["Primary Type"] == 'ARSON') ,'CRIMINAL TRESPASS').otherwise(dataset['Primary Type']))


In [0]:
primaryCount = dataset.groupby('Primary Type').count()
primaryCount.orderBy('count', ascending=False).show()

In [0]:
print('Distinct primary types after preprocessing '+ str(primaryCount.distinct().count()))

**Extract Day, Month, Week from Date Column**

In [0]:
dataset = dataset.withColumn("Day", F.split(dataset.Date, " ")[0])
dataset = dataset.withColumn("Day", F.to_date(dataset.Day, "MM/dd/yyyy"))
dataset = dataset.withColumn("Month", F.month(dataset.Day))
dataset = dataset.withColumn("WeekDay", F.dayofweek(dataset.Day))
dataset = dataset.withColumn("Year", F.year(dataset.Day))
dataset = dataset.na.drop()
dataset = dataset.drop('Date')
dataset = dataset.drop('Day')
dataset.head(5)

Ignore Latitude and Longitude outside Chichago

In [0]:
dataset=dataset\
        .filter((dataset["Latitude"] < 45)
             & (dataset["Latitude"] > 40)
             & (dataset["Longitude"] < -85)
             & (dataset["Longitude"] > -90))

**Dropping Null values in the data.**

In [0]:
dataset=dataset.na.drop()

Data conversion to Pandas DataFrame for KNN

In [0]:
data = dataset.toPandas()

In [0]:
# canalso get preprocessed data
#data = pd.read_csv('/content/gdrive/My Drive/BigData/preprocessed.csv')

In [ ]:
data.columns

# Data Visualization

In [0]:
primaryVisualizeDF = data.groupby('Primary Type').size()

In [0]:
primaryVisualizeDF.plot.pie(figsize = (15,15))
plt.xlabel('Primary Crime Type', fontsize=10, color = 'red')
plt.legend()

# Preprocessing - specific to KNN model

In [0]:
data = data.replace(to_replace='None', value=np.nan).dropna()

**Label Encoding categorical variables**

In [0]:
# create an instance of label encoder for categorical variables
labelEncoder = LabelEncoder()

In [ ]:
primaryEncoded = labelEncoder.fit_transform(data['Primary Type'])
primaryEncoded

In [0]:
data['Primary Type'] = primaryEncoded

In [0]:
# create an instance of label encoder for categorical variables
labelEncoder2 = LabelEncoder()

In [ ]:
locDiscEncoded = labelEncoder2.fit_transform(data['Location Description'])
locDiscEncoded

In [0]:
data['Location Description'] = locDiscEncoded

In [0]:
# create an instance of label encoder for categorical variables
labelEncoder3 = LabelEncoder()

In [ ]:
blockEncoded = labelEncoder2.fit_transform(data['Location Description'])
blockEncoded

In [0]:
data['Block'] = locDiscEncoded

Use PCA to combine features

In [0]:
from sklearn.decomposition import PCA

In [0]:
location = data[['Latitude','Longitude']]
pca = PCA(n_components=1,random_state=123)

In [0]:
loc = pca.fit_transform(location)

In [0]:
data['Location'] = loc

In [ ]:
data.head(5)

# Feature selection

Checking importance of features using an ExtraTreesClassifier

In [0]:
#taking a fraction of data
data_sample = data.sample(frac=0.1, random_state=123)

In [0]:

from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
model = ExtraTreesClassifier()

In [0]:

X = data_sample.loc[:, data.columns != 'Primary Type']

model.fit(X,data_sample['Primary Type'])

In [0]:
print(model.feature_importances_) #use inbuilt class feature_importances of tree based classifiers

In [0]:
#plot graph of feature importances for better visualization
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
#plot for top 10 features
feat_importances.nlargest(10).plot(kind='barh')
plt.show()

Find corelation between data to find dependency between data columns

In [0]:
corelation = data.corr()
corelation

In [0]:
sns.heatmap(corelation)

**Based on the above outputs, we have decided that below selected columns are corelated and best params for our KNN model**

In [0]:
selected_columns = ['Latitude','Longitude','Location Description','Block','WeekDay']
#selected_columns = ['Latitude','Longitude']

# Splitting data in train and test

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data[selected_columns], data['Primary Type'], test_size=0.3) # 70% training and 30% test

# Building initial KNN Classifier

**Initially we will build our classifier with 5 neighbours**

In [0]:
knn_5 = KNeighborsClassifier(n_neighbors=5) 
knn_5.fit(X_train,y_train)
pred_train = knn_5.predict(X_train)
pred_i = knn_5.predict(X_test)
print('Test accuracy ', metrics.accuracy_score(y_train, pred_train))
print('Accuracy ', metrics.accuracy_score(y_test, pred_i))
print('F1 score ', metrics.f1_score(y_test, pred_i, average = 'weighted'))

We will deploy Elbow method to get optimum number of neighbours

In [0]:
error_rate = []

krange = range(10,50,5)
for i in krange:
 
 knn = KNeighborsClassifier(n_neighbors=i, metric='manhattan', weights = 'uniform',n_jobs= -1) # haversine - if only lat long
 knn.fit(X_train,y_train)
 pred_train = knn.predict(X_train)
 pred_i = knn.predict(X_test)
 error_rate.append(np.mean(pred_i != y_test))
 print('Neighbours ',i)
 print('Test accuracy ', metrics.accuracy_score(y_train, pred_train))
 print('Accuracy ', metrics.accuracy_score(y_test, pred_i))
 print('F1 score ', metrics.f1_score(y_test, pred_i, average = 'weighted'))

In [0]:
plt.figure(figsize=(10,6))

plt.plot(krange,error_rate, color= 'blue', linestyle= 'dashed', marker= 'o', markerfacecolor='red', markersize=1)
plt.title('Error Rate vs. K Value')
plt.xlabel('K')
plt.ylabel('Error Rate')

K = 25, optimum

# Grid Search/ Random Search -KNN

**To find optimal paramteres for KNN Model, we will implement Grid search.**

In [0]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [0]:
KNeighborsClassifier().get_params().keys()

In [0]:
grid_params = {
    'weights' : ['uniform', 'distance'],
    'metric' : ['manhattan','jaccard'] #haversine - if only lat long
}

In [0]:
randomSearch = RandomizedSearchCV(
    KNeighborsClassifier(25),
    grid_params,
    verbose = 1,
    cv =3,
    random_state = 123
    )


In [0]:
#takes very long time
rs_results = randomSearch.fit(X_train_gs, y_train_gs)

In [0]:
rs_results.best_score_

In [0]:
rs_results.best_params_

# Oversmapling -SMOTE to balance dataset

In [0]:
from imblearn.over_sampling import SMOTE

In [0]:
# Oversample 'Majority'
sm = SMOTE('minority', random_state=123)

In [0]:
X_train_res, y_train_res = sm.fit_sample(X_train, y_train)

In [0]:
#convert back to Dataframe
X_train_updated = pd.DataFrame(X_train_res, columns= selected_columns)
y_train_updated = pd.Series(y_train_res)

In [0]:
target_counts = y_train_updated.value_counts()
target_counts

In [0]:
knn = KNeighborsClassifier(n_neighbors=25, metric='manhattan', weights = 'uniform')

In [0]:
knn.fit(X_train_updated,y_train_updated)

In [0]:
pred_i = knn.predict(X_test)

In [0]:
print('Accuracy ', metrics.accuracy_score(y_test, pred_i))
print('F1 score ', metrics.f1_score(y_test, pred_i, average = 'weighted'))

Oversampling the Minority classed helped with an increase in accuracy and F1 score

# Undersampling - Cluster Centroids

In [0]:
from imblearn.under_sampling import ClusterCentroids

In [0]:
# Undersample 'Majority'
cc = ClusterCentroids(sampling_strategy = 'majority')

In [0]:
X_cc, y_cc = cc.fit_sample(X_train, y_train)

In [0]:
#convert back to Dataframe
X_train_updated = pd.DataFrame(X_cc, columns= selected_columns)
y_train_updated = pd.Series(y_cc)

In [0]:
target_counts = y_train_updated.value_counts()
target_counts

In [0]:
knn = KNeighborsClassifier(n_neighbors=25, metric='manhattan', weights = 'uniform')

In [0]:
knn.fit(X_train_updated,y_train_updated)

In [0]:
pred_i = knn.predict(X_test)

In [0]:
print('Accuracy ', metrics.accuracy_score(y_test, pred_i))
print('F1 score ', metrics.f1_score(y_test, pred_i, average = 'weighted'))

Undersampling using cluster centroids does not help much in high dimensional data

# Random Sampling

In [0]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

In [0]:
# define oversampling strategy
over = RandomOverSampler(sampling_strategy= 'minority') 

In [0]:
# fit and apply the transform
X_random, y_random = over.fit_resample(X_train, y_train)

In [0]:
#convert back to Dataframe
X_train_updated = pd.DataFrame(X_random, columns= selected_columns)
y_train_updated = pd.Series(y_random)

In [0]:
target_counts_random = y_train_updated.value_counts()
target_counts_random

In [0]:
knn = KNeighborsClassifier(n_neighbors=25, metric='manhattan', weights = 'uniform')

In [0]:
knn.fit(X_train_updated,y_train_updated)

In [0]:
pred_i = knn.predict(X_test)

In [0]:
print('Accuracy ', metrics.accuracy_score(y_test, pred_i))
print('F1 score ', metrics.f1_score(y_test, pred_i, average = 'weighted'))

OverSampling minority class improves model 

In [0]:
# define undersampling strategy
under = RandomUnderSampler(sampling_strategy= 'majority')

In [0]:
# fit and apply the transform
X_train_updated, y_train_updated = under.fit_resample(X_train, y_train)

In [0]:
y_train_updated = pd.Series(y_train_updated)
target_counts_random = y_train.value_counts()
target_counts_random

In [0]:
knn = KNeighborsClassifier(n_neighbors=25, metric='manhattan', weights = 'uniform')

In [0]:
knn.fit(X_train_updated,y_train_updated)

In [0]:
pred_i = knn.predict(X_test)

In [0]:
print('Accuracy ', metrics.accuracy_score(y_test, pred_i))
print('F1 score ', metrics.f1_score(y_test, pred_i, average = 'weighted'))

Oversampling does not have significant impact on model

# Ensemble models

Here we will try ensemble of Various classification models 

In [0]:
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

In [0]:
#KNN with - 25 neighbours
knn = KNeighborsClassifier(25, metric='manhattan', weights = 'uniform',n_jobs= -1)

#random forest with 75 Decision Trees
rf = RandomForestClassifier(10, max_depth = 10, n_jobs= -1)

#Support Vector Classifier
svm = SVC(kernel = 'linear', random_state = 123, probability=True)

In [ ]:
# can using previopusly trained KNN as well
knn.fit(X_train1,y_train1)

In [ ]:
rf.fit(X_train,y_train)

In [ ]:
rf.score(X_test, y_test)

In [ ]:
svm.fit(X_train1,y_train1)

In [ ]:
svm.score(X_test, y_test)

In [0]:
#create a dictionary of our models
estimators=[('knn', knn), ('rf', rf), ('svm', svm)]
#create our voting classifier, inputting our models
ensemble = VotingClassifier(estimators, voting='hard')

In [0]:
#fit model to training data
ensemble.fit(X_train, y_train)
#test our model on the test data
ensemble.score(X_test, y_test)